In [1]:
import datetime
import logging
import os
import numpy as np
from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)
import matplotlib.pyplot as plt
## Only when developing with VSCode in my repo 
#os.chdir('/home/nicolas/codeRepo/OceanPlatformControl') # here in Azure
# os.chdir('/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl') # here in WSL
# print(os.getcwd())
##
import pandas as pd
from ocean_navigation_simulator.utils.units import Distance
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2

In [ ]:
wandb_log_name = "HC_HC_30_platforms_epsilon"

### Define Filenames

In [2]:
path_to_files = 'generated_media/metrics_csv_results/'
#file_containing_pb = 'problemsGOM.csv'
pb_name_hj_naive = 'multi_ag_30_HC_HC_hj_naive_1123.csv'
pb_name_reactive = 'multi_ag_30_HC_HC_reactive_control_1106.csv'
pb_name_flocking = 'multi_ag_30_HC_HC_epsil_0_25_flocking_1052.csv'

### Plot problem distribution

In [3]:
# TODO

### Extract metrics from csv files

In [3]:
metrics_hj_naive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_hj_naive}")
metrics_hj_reactive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_reactive}")
metrics_hj_flocking_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_flocking}")

+ Make sure we have the (same) indices and total number of missions for the panda dataframes

In [4]:
# Find the common mission_nr values
common_mission_nr = set(metrics_hj_naive_unfiltered['mission_nr']).intersection(set(metrics_hj_reactive_unfiltered['mission_nr'])).intersection(set(metrics_hj_flocking_unfiltered['mission_nr']))
# Filter the rows in each dataframe that have matching mission_nr values
metrics_hj_naive = metrics_hj_naive_unfiltered[metrics_hj_naive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_reactive = metrics_hj_reactive_unfiltered[metrics_hj_reactive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_flocking = metrics_hj_flocking_unfiltered[metrics_hj_flocking_unfiltered['mission_nr'].isin(common_mission_nr)]

+ Sort by ascending missions nr order

In [5]:
metrics_hj_naive = metrics_hj_naive.sort_values(['mission_nr'])
metrics_hj_reactive= metrics_hj_reactive.sort_values(['mission_nr'])
metrics_hj_flocking = metrics_hj_flocking.sort_values(['mission_nr'])

+ Index by mission nr to get the same order in the panda dataframe between controller metrics

In [6]:
metrics_hj_naive.set_index('mission_nr', inplace=True)
metrics_hj_reactive.set_index('mission_nr', inplace=True)
metrics_hj_flocking.set_index('mission_nr', inplace=True)

+ Get Non Feasible Missions 

In [7]:
non_feas_idx = metrics_hj_naive.index[metrics_hj_naive["Reaching_target"] < 1]
ratio_non_feas_miss = len(non_feas_idx)/len(metrics_hj_naive)*100
print("Number of infeasible missions: ", len(non_feas_idx), f"corresponds to {ratio_non_feas_miss} %")

Number of infeasible missions:  75 corresponds to 7.575757575757576 %


+ Drop infeasible missions on HC

In [8]:
metrics_hj_naive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_reactive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_flocking.drop(non_feas_idx, axis=0, inplace=True)

In [9]:
import plotly.express as px
import plotly.graph_objects as go

## Box Plots

In [10]:
from plotly.offline import  init_notebook_mode
init_notebook_mode(connected=True)

+ Beta Index

In [11]:
beta_index_avg_dic = {"hj_naive": metrics_hj_naive["Average Beta Index"],
                      "hj_reactive": metrics_hj_reactive["Average Beta Index"],
                      "flocking": metrics_hj_flocking["Average Beta Index"]}
beta_index_avg_df = pd.DataFrame(beta_index_avg_dic)
fig_beta = px.box(beta_index_avg_df)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Beta Index over Missions"
)
fig_beta.show()

+ Mean minimum distance to target among all platforms km

In [12]:
mean_min_dist_dict = {"hj_naive": Distance(deg=metrics_hj_naive["Mean minimum distance to target among all platforms in deg"]).km,
                      "hj_reactive": Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km,
                      "flocking":Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km}
idx_list = []
mean_min_dist_df = pd.DataFrame(mean_min_dist_dict)
fig_min_dist = px.box(mean_min_dist_df, log_y=True)
mean_trace = go.Scatter(
    x=mean_min_dist_df.columns,
    y=mean_min_dist_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_min_dist.add_trace(mean_trace)
# Add custom x and y labels
fig_min_dist.update_layout(
    xaxis_title="controllers",
    yaxis_title=" [km]",
    title = {'text': "Mean minimum distance to target among all platforms"}
)
fig_min_dist.show()

+ Mean maximum correction from optimal control degrees

In [13]:
opt_ctrl_correction_dict = {"hj_naive": metrics_hj_naive["Mean maximum correction from optimal control degrees"],
                      "hj_reactive": metrics_hj_reactive["Mean maximum correction from optimal control degrees"],
                      "flocking": metrics_hj_flocking["Mean maximum correction from optimal control degrees"]}
opt_ctrl_correction_df = pd.DataFrame(opt_ctrl_correction_dict)
fig_opt_ctrl_correction = px.box(opt_ctrl_correction_df )
# Add custom x and y labels
fig_opt_ctrl_correction.update_layout(
    xaxis_title="controllers",
    yaxis_title="degrees",
    title = {'text': "Control direction correction from optimal input"}
)
fig_opt_ctrl_correction.show()

+ Isolated Platform Metric

In [14]:
ipm_dict= {"hj_naive": metrics_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": metrics_hj_reactive["Isolated_platform_metric"],
                      "flocking": metrics_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    xaxis_title="controllers",
    yaxis_title="IPM",
    title = {'text': "Isolated_platform_metric"}
)
fig_ipm.show()

### Get Metrics summary as a dict:

In [15]:
def get_metrics_summary(metrics_file:pd.DataFrame, ctrl_name: str)->dict:
    return {
    "ctrl_name": ctrl_name,
    "avg_beta_index":metrics_file["Average Beta Index"].mean(),
    "avg_isolated_pltf_integral_hj_naive": metrics_file["Isolated_platform_metric"].mean(),
    "avg_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].mean(),
    "avg_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].mean()).km,
    "avg_nb_platfroms_reaching_target": metrics_file["Reaching_target"].mean(),
    "nb_missions_with_isolated_pltf": np.sum(metrics_file["Isolated_platform_metric"] > 0),
    "nb_missions_with_collisions": np.sum(metrics_file["Number_of_collision"] > 0),
    "nb_missions_success": np.sum(metrics_file["Mission_success"]==1),
}

  + HJ Multi-Agent Naive

In [16]:
metrics_summary_hj_naive = get_metrics_summary(metrics_hj_naive, ctrl_name="hj_naive")
print(metrics_summary_hj_naive)
summary_hj_naive_df = pd.DataFrame(metrics_summary_hj_naive, index=[0])

{'ctrl_name': 'hj_naive', 'avg_beta_index': 6.531777082872695, 'avg_isolated_pltf_integral_hj_naive': 0.3497912871888281, 'avg_correction_from_opt_ctrl_deg': 0.0, 'avg_mean_min_dist_to_target_km': 0.0, 'avg_nb_platfroms_reaching_target': 1.0, 'nb_missions_with_isolated_pltf': 534, 'nb_missions_with_collisions': 864, 'nb_missions_success': 9}


+ HJ Multi-Agent Decentralized Reactive Control

In [17]:
metrics_summary_hj_reactive = get_metrics_summary(metrics_hj_reactive, ctrl_name='hj_reactive')
print(metrics_summary_hj_reactive)
summary_hj_naive_reactive = pd.DataFrame(metrics_summary_hj_reactive, index=[0])

{'ctrl_name': 'hj_reactive', 'avg_beta_index': 5.812443896943471, 'avg_isolated_pltf_integral_hj_naive': 0.17923202118329623, 'avg_correction_from_opt_ctrl_deg': 95.27882856698753, 'avg_mean_min_dist_to_target_km': 0.1178188988015108, 'avg_nb_platfroms_reaching_target': 0.9653916211293261, 'nb_missions_with_isolated_pltf': 343, 'nb_missions_with_collisions': 139, 'nb_missions_success': 379}


+ HJ Multi-Agent Flocking

In [18]:
metrics_summary_hj_flocking = get_metrics_summary(metrics_hj_flocking, ctrl_name='hj_flocking')
print(metrics_summary_hj_flocking)
summary_hj_naive_flocking= pd.DataFrame(metrics_summary_hj_flocking, index=[0])

{'ctrl_name': 'hj_flocking', 'avg_beta_index': 12.83221394653442, 'avg_isolated_pltf_integral_hj_naive': 0.04385836200499224, 'avg_correction_from_opt_ctrl_deg': 124.67531235311974, 'avg_mean_min_dist_to_target_km': 3.260872462895245, 'avg_nb_platfroms_reaching_target': 0.7957012750455374, 'nb_missions_with_isolated_pltf': 110, 'nb_missions_with_collisions': 538, 'nb_missions_success': 223}


## Confusion Matrices Plots

In [19]:
def plot_confusion_matrix(x, y, x_label, y_label, title_name:str):
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm

    confusion_matrix = pd.crosstab(x, y, rownames=[x_label], colnames=[y_label])
    fig = px.imshow(confusion_matrix, text_auto=True)
    fig.update_layout(
        title = {'text': title_name}
    )
    fig.show()
    # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, fmt='g')
    # # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, norm=LogNorm(), fmt='g') # if heat map is in norm
    # plt.tick_params(axis='both', which='major', labelbottom=False, bottom=False, top=True, labeltop=True)
    # ax.xaxis.set_label_position('top')
    # ax.plot
    # plt.title(title_name)
    # plt.show()
    return fig

###  HJ naive vs Decentralized Reactive Control

+  Missions with collisions

In [20]:
fig_conf_react_coll = plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_reactive["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with collisions")

+ Missions with communication losses

In [21]:
fig_conf_react_comm_loss= plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_reactive["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [22]:
connect_objective_reactive = (metrics_hj_reactive["Isolated_platform_metric"]==0) & (metrics_hj_reactive["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_react_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_reactive , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Reactive Control', title_name= 'Missions without collisions and with maintained connectivity')

+  Missions success counts

In [23]:
fig_react_success  = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_reactive["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive', title_name="Mission Success")

###  HJ naive vs Flocking

+  Missions with collisions

In [24]:
fig_conf_flocking_coll= plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_flocking["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with collisions")

+ Missions with communication losses

In [25]:
fig_conf_flocking_comm_loss = plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_flocking["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [26]:
connect_objective_flocking = (metrics_hj_flocking["Isolated_platform_metric"]==0) & (metrics_hj_flocking["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_flocking_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_flocking , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Flocking Control', title_name= 'Missions without collisions and with maintained connectivity')

In [27]:
index_list = metrics_hj_flocking.index[(connect_objective_reactive==True) & (connect_objective_flocking==False)].tolist()


In [29]:
index_list

[285]

In [28]:
metrics_hj_reactive.loc[285]

KeyError: 285

+  Missions success counts

In [29]:
fig_flocking_success = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_flocking["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking', title_name="Mission Success")

## Log Data in WandB

In [30]:
import wandb

In [31]:
# Log metrics in WandB
os.environ["WANDB_API_KEY"] = "1f19232e6ccc9981a8a972bee18ba31a94644835"

wandb.init(
    # Set the project where this run will be logged
    project="Master_Thesis_Run_Summaries",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=wandb_log_name,
    # Track hyperparameters and run metadata
    config={
        "Number of infeasible missions": len(non_feas_idx),
        "Percentage of infeasible missions": ratio_non_feas_miss,
        "Number of missions feasible": len(metrics_hj_naive),
        "Ratio failing missions with HJ naive in terms of connectivity and collisions": 
        len(metrics_hj_naive.index[(connect_objective_hj_naive==False)].tolist())/len(metrics_hj_naive),
    },
    entity="nhoischen",
)
wandb.log({"Metrics_Summary": wandb.Table(dataframe=pd.concat([summary_hj_naive_df, summary_hj_naive_reactive, summary_hj_naive_flocking]))},
            commit=False)
wandb.log({'Beta Index': fig_beta,
          'Time-Integral-over-Isolated-Platforms': fig_ipm,
          'Platforms-Mean-Min-Distance-to-Target': fig_min_dist,
           'Deviation-from-Cptimal-Ctrl-Direction': fig_opt_ctrl_correction,
            },
           commit=False)
wandb.log({'Missions with collisions hj-reactive': fig_conf_react_coll,
         'Missions with collisions hj-flocking': fig_conf_flocking_coll,
         'Missions with communication losses hj-reactive': fig_conf_react_comm_loss,
         'Missions with communication losses hj-flocking': fig_conf_flocking_comm_loss,
         'Missions with maintained connectivity and no collisions hj-reactive': fig_react_maint,
         'Missions with maintained connectivity and no collisions hj-flocking': fig_flocking_maint,
         'Succesfull Missions hj-reactive': fig_react_success,
         'Succesfull Missions hj-flocking': fig_flocking_success}, commit=True)
wandb.finish()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nhoischen. Use `wandb login --relogin` to force relogin
